# Finding the nearest zip code

**Abstract**: Given a list of 'start points', find the nearest neighbor in a second list of 'end points'<br><br>
**Real World Problem Example**: Marketers could use nearest zip code distancing for approximating the nearest store (end points) for a group of prospects/targets/customjers (start points)<br>
**Problem Details**: Input data are pickled dataframes that contain identifcally formatted columns of 'zip code', 'latitude', and 'longitude'. The 'latitude and 'longitude' geo-coordinates are centroids for each zip code, and each zip code has only one unique set of geo-coordinates.<br><br>
**Approach**: Below is a brute force algorithmic approach. Leveraging numpy's vecotorization capabilites whever possible.<br>
**Heuristic**: Identify the intersection of zip codes between the 'start points' and 'end points', because these 'start points' are already nearest to own zip codes.<br>
**Algorithm**: Deploys and leverages numpy's vectorizations capabilities to calculate and find the min end points for each start point.

In [1]:
import numpy as np
import pandas as pd

# separates the intersection of two arrays of zipcodes
# input of an input array of zipcodes and lookup array of zipcodes
# returns a tuple of inverse boolean arrays that denote matched and unmatched input zipcodes
def match_zipcode(input_zipcodes, lookup_zipcodes):
    match_boolean = input_zipcodes.isin(lookup_zipcodes)
    return match_boolean, ~match_boolean

# input of a start point and an array of end points
# returns an array of calculated arc distances
degrees_to_radians = np.pi/180.0
def arc_distance_calculation(start_point, end_points):
    phi1 = (90.0 - start_point[0])*degrees_to_radians
    phi2 = (90.0 - end_points[:, 0])*degrees_to_radians

    theta1 = start_point[1]*degrees_to_radians
    theta2 = end_points[:, 1]*degrees_to_radians
    
    cos = (np.sin(phi1)*np.sin(phi2)*np.cos(theta1 - theta2) + np.cos(phi1)*np.cos(phi2))
    cos = np.minimum(1, cos)
    cos = np.maximum(-1, cos)
    return np.arccos(cos)

# input of a start point and an array of end points
# returns the idx and min distance of nearest end point to start point
def min_end_point_distancing(start_point, end_points):
    distances = np.array(arc_distance_calculation(start_point, end_points))
    idx = distances.argmin()
    return np.array([idx, distances[idx]])

# input of an array of start points and array of end points
# returns the idx and min distance of nearest end point for every start point
def loop_min_end_point_distancing(start_points, end_points):
    _start_points = start_points.loc[:, ['lat','lng']].values 
    return [min_end_point_distancing((x[0], x[1]), end_points.loc[:, ['lat','lng']].values) for x in _start_points]
    
# input of a dataframe of start points and dataframe of end points
# returns a dataframe of start points with nearest end points appended as columns
def get_nearest_zipcode(start_points, end_points, sample=0):
#     used for testing and debugging
    if sample > 0:
        start_points = start_points.sample(sample, random_state=sample)
        end_points = end_points.sample(sample, random_state=sample)
    
#     splitting matched and unmatched zip codes of start points and end points
    matched, unmatched = match_zipcode(start_points.zip, end_points.zip)
    
#     creating output fields for matched zip codes
    matched = start_points[matched]
    matched.loc[:, 'nearest_zip'] = matched['zip']
    matched.loc[:, 'nearest_lat'] = matched['lat']
    matched.loc[:, 'nearest_lng'] = matched['lng']
    matched.loc[:, 'nearest_distance'] = 0
        
#     algorithmic min distance search for unmatched zip codes
    unmatched = start_points[unmatched]
    _nearest_idx_distance = np.stack(loop_min_end_point_distancing(unmatched, end_points))
    nearest_end_points = end_points.iloc[_nearest_idx_distance[:,0]].rename(columns={'zip':'nearest_zip',
                                                                                     'lat':'nearest_lat',
                                                                                     'lng':'nearest_lng'})
    nearest_end_points.loc[:, 'nearest_distance'] = _nearest_idx_distance[:,1] * 3959 # earth's radius in miles
    unmatched = pd.concat([unmatched.reset_index(drop=True), nearest_end_points.reset_index(drop=True)], axis=1)
    
    return pd.concat([matched, unmatched], ignore_index=True)

In [2]:
# importing dataframes of 'start' and 'end' points
start_points = pd.read_pickle('start_zip_points.pkl').dropna()
start_points = start_points[~start_points.zip.duplicated()]

end_points = pd.read_pickle('end_zip_points.pkl').dropna()
end_points = end_points[~end_points.zip.duplicated()]

# 'start' and 'end' dataframes contain three identical columns
start_points.head()

,zip,lat,lng
0,10965,41.0618,-74.0128
1,32539,30.7748,-86.4631
2,10314,40.5992,-74.1657
3,54115,44.4046,-88.0920
4,53546,42.6527,-88.9492


In [3]:
# showing the number of iterations for these two sets of start and end points
matched, unmatched = match_zipcode(start_points.zip, end_points.zip)

print('total_start_points, end_points:', (len(unmatched), len(end_points)))
print('unmatched_start_points, end_points:', (sum(unmatched), len(end_points)))
print('algorithm iterations excluding matched zip codes:', sum(unmatched) * len(end_points))

total_start_points, end_points: (16438, 4349)
unmatched_start_points, end_points: (12220, 4349)
algorithm iterations excluding matched zip codes: 53144780


In [5]:
import timeit
start = timeit.default_timer()
results = get_nearest_zipcode(start_points, end_points)
end = timeit.default_timer()

print('runtime:', end-start)

runtime: 12.757086266999977


In [6]:
results.head()

,zip,lat,lng,nearest_zip,nearest_lat,nearest_lng,nearest_distance
0,10965,41.0618,-74.0128,10965,41.0618,-74.0128,0.0
1,32539,30.7748,-86.4631,32539,30.7748,-86.4631,0.0
2,10314,40.5992,-74.1657,10314,40.5992,-74.1657,0.0
3,54115,44.4046,-88.0920,54115,44.4046,-88.0920,0.0
4,16137,41.2301,-80.2384,16137,41.2301,-80.2384,0.0


In [7]:
results.tail()

,zip,lat,lng,nearest_zip,nearest_lat,nearest_lng,nearest_distance
16433,60603,41.8802,-87.6255,60607,41.8750,-87.6516,1.390038
16434,61769,40.8820,-88.3993,61764,40.8823,-88.6282,11.958149
16435,43469,41.4595,-83.3632,49270,41.8697,-83.6833,32.808367
16436,99362,46.0893,-118.3074,54730,45.0200,-91.7234,1282.361769
16437,76691,31.7782,-97.0963,76684,31.7125,-97.1164,4.690835


## Take-aways:
I think that this algorithms performance could possibly be further improved by replacing the singular pd.DataFrame.apply step with pure numpy vectorization, which would yield a numpy ndarray of shape (x, y, z) spanning three dimensions. TBD.